# Feature Engineering

This notebook extends the baseline gradient boosting approach with more comprehensive feature engineering. Multiple fingerprint types, descriptors, 3D conformational features, and transformer embeddings are explored to improve predictive performance.

Target-specific feature combinations are optimised through systematic evaluation, balancing feature diversity with overfitting prevention on the limited dataset.

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import importnb
import joblib
from rdkit.Chem import MolFromSmiles, Descriptors
from rdkit.Chem.rdFingerprintGenerator import *
from rdkit.Chem.rdMolDescriptors import GetMACCSKeysFingerprint
from rdkit.Chem.GraphDescriptors import *
from rdkit.Chem.Crippen import MolLogP, MolMR
from rdkit.Chem.Lipinski import NumHDonors, NumHAcceptors
from rdkit.Chem.rdMolDescriptors import CalcNumRotatableBonds, CalcTPSA
from rdkit.Chem import AllChem, Descriptors3D
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import KFold

In [ ]:
grad_boosting = importnb.Notebook.load_file("01_gradient_boosting.ipynb", include_non_defs=False)

In [ ]:
def validate_features(feature: pd.Series) -> pd.Series:
    """Replaces infinite and infinite-like values with NaN."""
    feature = pd.to_numeric(feature, errors='coerce')
    feature = feature.replace([np.inf, -np.inf], np.nan)
    
    with np.errstate(over='ignore'):
        feature = feature.mask(feature.abs() > 1e10, np.nan)
    
    return feature

In [ ]:
def create_fingerprint_getter(generator):
    """Creates a callable that returns a fingerprint from a SMILES string.

    Args:
        generator: RDKit fingerprint generator with GetFingerprint method.
    """
    def get_fingerprint(smile: str) -> pd.Series:
        """Returns validated fingerprint from the configured generator."""
        mol = MolFromSmiles(smile)
        fingerprint = pd.Series(list(generator.GetFingerprint(mol)))
        return validate_features(fingerprint)

    return get_fingerprint

In [ ]:
get_morgan_2_fp = create_fingerprint_getter(GetMorganGenerator(radius=2, fpSize=256))
get_morgan_5_fp = create_fingerprint_getter(GetMorganGenerator(radius=5, fpSize=256))
get_rdkit_fp = create_fingerprint_getter(GetRDKitFPGenerator(fpSize=256))
get_atom_pair_fp = create_fingerprint_getter(GetAtomPairGenerator(fpSize=256))
get_topological_torsion_fp = create_fingerprint_getter(GetTopologicalTorsionGenerator(fpSize=256))

In [ ]:
def get_maccs_keys_fp(smiles_string: str) -> pd.Series:
    """Extracts MACCS keys fingerprint from SMILES string."""

    mol = MolFromSmiles(smiles_string)
    fingerprint = pd.Series(list(GetMACCSKeysFingerprint(mol)))

    return validate_features(fingerprint)

In [ ]:
def get_rdkit_descriptors(smiles_string: str) -> pd.Series:
    """Computes all RDKit molecular descriptors."""
    mol = MolFromSmiles(smiles_string)
    features = pd.Series([desc_fn(mol) for name, desc_fn in Descriptors.descList]) # type: ignore

    return validate_features(features)

In [ ]:
def get_rdkit_3d_features(smiles_string: str) -> pd.Series:
    """Computes 3D geometric descriptors from optimised molecular conformation.
    
    Embeds molecule in 3D space and optimises geometry before computing
    shape-based descriptors like asphericity and radius of gyration.
    """
    smiles_processed = smiles_string.replace('*', '[H]')
    mol = MolFromSmiles(smiles_processed)

    try:
        if mol is None:
            raise ValueError("Failed to parse molecule.")
        mol = Chem.AddHs(mol)
        result = AllChem.EmbedMolecule(mol, randomSeed=42)  # type: ignore

        if result != 0:
            raise ValueError("Embedding failed")
        try:
            AllChem.MMFFOptimizeMolecule(mol)  # type: ignore
        except:
            pass

        return validate_features(pd.Series({
            "asphericity": Descriptors3D.Asphericity(mol),  # type: ignore
            "eccentricity": Descriptors3D.Eccentricity(mol),  # type: ignore
            "inertial_shape_factor": Descriptors3D.InertialShapeFactor(mol),  # type: ignore
            "radius_of_gyration": Descriptors3D.RadiusOfGyration(mol),  # type: ignore
            "spherocity_index": Descriptors3D.SpherocityIndex(mol)  # type: ignore
        }))
    except ValueError:
        return pd.Series({
            "asphericity": np.nan,
            "eccentricity": np.nan,
            "inertial_shape_factor": np.nan,
            "radius_of_gyration": np.nan,
            "spherocity_index": np.nan
        })

In [ ]:
def mol_to_networkx(mol):
    """Converts RDKit molecule to NetworkX graph representation."""
    G = nx.Graph()

    for atom in mol.GetAtoms():
        G.add_node(
            atom.GetIdx(),
            atomic_num=atom.GetAtomicNum(),
            formal_charge=atom.GetFormalCharge(),
            hybridization=atom.GetHybridization()
        )

    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())

    return G

In [ ]:
def safe_diameter(G):
    """Calculates graph diameter, handling disconnected graphs gracefully."""
    try:
        if nx.is_connected(G):
            return nx.diameter(G)
        else:
            # Return diameter of largest connected component
            largest_cc = max(nx.connected_components(G), key=len)
            subgraph = G.subgraph(largest_cc)

            return nx.diameter(subgraph) if len(largest_cc) > 1 else 0
    except:
        return np.nan

In [ ]:
def safe_radius(G):
    """Calculates graph radius, handling disconnected graphs gracefully."""
    try:
        if nx.is_connected(G):
            return nx.radius(G)
        else:
            # Return radius of largest connected component
            largest_cc = max(nx.connected_components(G), key=len)
            subgraph = G.subgraph(largest_cc)

            return nx.radius(subgraph) if len(largest_cc) > 1 else 0
    except:
        return np.nan

In [ ]:
def safe_average_clustering(G):
    """Calculates average clustering coefficient with error handling."""
    try:
        return nx.average_clustering(G) if G.number_of_nodes() > 0 else 0
    except:
        return np.nan

In [ ]:
def safe_avg_centrality(G, centrality_func):
    """Calculates average centrality measure with error handling.
    
    Args:
        centrality_func: NetworkX centrality function to apply.
    """
    try:
        if G.number_of_nodes() == 0:
            return 0
        centrality_dict = centrality_func(G)

        return np.mean(list(centrality_dict.values()))
    except:
        return np.nan

In [ ]:
def calculate_randic_index(mol):
    """Calculates Randić connectivity index for molecular graph."""
    try:
        randic = 0.0
        for bond in mol.GetBonds():
            atom1 = bond.GetBeginAtom()
            atom2 = bond.GetEndAtom()
            deg1 = atom1.GetDegree()
            deg2 = atom2.GetDegree()
            if deg1 > 0 and deg2 > 0:
                randic += 1.0 / np.sqrt(deg1 * deg2)

        return float(randic)
    except:
        return np.nan

In [ ]:
def get_graph_features(smiles_string: str) -> pd.Series:
    """Extracts comprehensive graph-based molecular features.
    
    Computes connectivity indices, topological descriptors, ring properties,
    and physicochemical properties from molecular graph structure.
    """
    mol = MolFromSmiles(smiles_string)

    features = pd.Series({
        "kappa1": Kappa1(mol),
        "chi0v": Chi0v(mol),
        "chi1v": Chi1v(mol),
        "chi0n": Chi0n(mol),
        'chi1n': Chi1n(mol),
        
        "hallkieralpha": HallKierAlpha(mol),
        "balabanj": BalabanJ(mol),
        
        "num_heavy_atoms": mol.GetNumHeavyAtoms(),
        
        "num_aromatic_rings": len([x for x in mol.GetRingInfo().AtomRings() if all(mol.GetAtomWithIdx(i).GetIsAromatic() for i in x)]),
        "num_aliphatic_rings": len([x for x in mol.GetRingInfo().AtomRings() if not all(mol.GetAtomWithIdx(i).GetIsAromatic() for i in x)]),
        "num_rings": len(mol.GetRingInfo().AtomRings()),
        "largest_ring_size": max([len(x) for x in mol.GetRingInfo().AtomRings()]) if mol.GetRingInfo().AtomRings() else 0,
        
        "num_hbd": NumHDonors(mol),
        "num_hba": NumHAcceptors(mol),
        "num_rotatable_bonds": CalcNumRotatableBonds(mol),
        "tpsa": CalcTPSA(mol),
        "logp": MolLogP(mol),
        "molmr": MolMR(mol),
        
        "randic_index": calculate_randic_index(mol)
    })

    return validate_features(features)

In [ ]:
def get_nx_graph_features(smiles_string: str) -> pd.Series:
    """Extracts NetworkX graph topology features from molecular structure."""
    mol = MolFromSmiles(smiles_string)
    G = mol_to_networkx(mol)

    features = pd.Series({
        "diameter": safe_diameter(G),
        "average_clustering": safe_average_clustering(G),
        "density": nx.density(G) if G.number_of_nodes() > 0 else 0,
        "avg_degree_centrality": safe_avg_centrality(G, nx.degree_centrality),
    })

    return validate_features(features)

In [ ]:
def mean_pooling(model_output, attention_mask):
    """Applies attention-masked mean pooling to transformer outputs.
    
    Implementation from Hugging Face model documentation.
    
    Args:
        attention_mask: Binary mask indicating valid tokens.
    """
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
def set_up_model(model_name: str):
    """Loads pre-trained tokeniser and model from Hugging Face."""
    tokeniser = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

    return tokeniser, model

In [ ]:
def process_batches(smiles_list, model_name: str, pooling_fn, device: str = "mps", batch_size: int = 32):
    """Processes SMILES strings through transformer model in batches.
    
    Args:
        pooling_fn: Function to pool token embeddings into sentence embeddings.
        batch_size: Number of samples per batch.
    """
    tokeniser, model = set_up_model(model_name)
    model = model.to(device)
    tokens = tokeniser(smiles_list, padding=True, truncation=True, return_tensors="pt")

    smile_dataset = TensorDataset(tokens["input_ids"], tokens["attention_mask"])
    dataloader = DataLoader(smile_dataset, batch_size=batch_size)
    
    embeddings = []
    for input_ids, attention_mask in tqdm(dataloader, desc=model_name):
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            embedding = pooling_fn(outputs, attention_mask)
            embeddings.append(embedding.cpu())
    
    return pd.DataFrame(torch.cat(embeddings).numpy())

In [ ]:
def get_embedding_fingerprints(smiles_strings: pd.Series) -> tuple:
    """Generates transformer-based molecular embeddings using polyBERT, ChemBERTa, and MoLFormer."""
    smiles_list = smiles_strings.tolist()

    polyBERT_fingerprints = process_batches(
        smiles_list,
        "kuelumbus/polyBERT",
        mean_pooling,
    )

    chemBERTa_fingerprints = process_batches(
        smiles_list,
        "DeepChem/ChemBERTa-77M-MLM",
        lambda outputs, mask: outputs.last_hidden_state[:, 0, :]
    )

    MoLFormer_fingerprints = process_batches(
        smiles_list,
        "ibm/MoLFormer-XL-both-10pct",
        lambda outputs, mask: outputs.pooler_output,
        device="cpu"
    )

    return polyBERT_fingerprints, chemBERTa_fingerprints, MoLFormer_fingerprints

In [ ]:
train_data = grad_boosting.load_train_data()
smiles = train_data["SMILES"]

In [ ]:
rdkit_descriptors = smiles.apply(get_rdkit_descriptors)

nx_graph_features = smiles.apply(get_nx_graph_features)
rdkit_3d_features = smiles.apply(get_rdkit_3d_features)
graph_features = smiles.apply(get_graph_features)

morgan_2_fps = smiles.apply(get_morgan_2_fp)
morgan_5_fps = smiles.apply(get_morgan_5_fp)
rdkit_fps = smiles.apply(get_rdkit_fp)
atom_pair_fps = smiles.apply(get_atom_pair_fp)
topological_torsion_fps = smiles.apply(get_topological_torsion_fp)
maccs_fps = smiles.apply(get_maccs_keys_fp)

polyBERT_fps, chemBERTa_fps, MoLFormer_fps = get_embedding_fingerprints(smiles)

In [ ]:
def combine_features(train_data: pd.DataFrame, *feature_sets) -> tuple:
    """Combines multiple feature sets with variance and correlation filtering.
    
    Args:
        *feature_sets: Variable number of feature DataFrames to combine.
    """
    all_features = []
    for i, feature_set in enumerate(feature_sets):
        feature_set_renamed = feature_set.add_suffix(f'_set{i}')
        all_features.append(feature_set_renamed)

    features = pd.concat(all_features, axis=1)
    mask = grad_boosting.get_feature_mask(features)
    train_data = (pd.concat([train_data, features.loc[:, mask]], axis=1)
                  .drop(columns=["SMILES"]))

    return train_data, mask

### Feature Selection

The set of features to be used for each property was determined in two parts: first, descriptors and fingerprints were tested in isolation and compared. With sample sizes as small as ~600, it's unlikely that complex relationships would be discovered, so exploring combinations is less beneficial.

Then, smaller features sets, which underperformed in step 1, were added to provide additional context, relying on domain knowledge. For example, Glass transition temperature (Tg) was likely to benefit from 3D features.

In [ ]:
tg_feats, tg_mask = combine_features(train_data, rdkit_descriptors, maccs_fps, morgan_2_fps, rdkit_3d_features)
ffv_feats, ffv_mask = combine_features(train_data, rdkit_descriptors, maccs_fps, morgan_2_fps, graph_features)
tc_feats, tc_mask = combine_features(train_data, rdkit_descriptors, morgan_5_fps, atom_pair_fps)
density_feats, density_mask = combine_features(train_data, rdkit_descriptors, maccs_fps, morgan_2_fps, rdkit_3d_features)
rg_feats, rg_mask = combine_features(train_data, rdkit_descriptors, maccs_fps, topological_torsion_fps, rdkit_3d_features)

In [ ]:
oof_preds = pd.DataFrame(index=train_data.index)
targets = ["Tg", "FFV", "Tc", "Density", "Rg"]
features = [tg_feats, ffv_feats, tc_feats, density_feats, rg_feats]

for target, feats in zip(targets, features):
    k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

    for i, (train_index, test_index) in enumerate(tqdm(
        k_fold.split(train_data),
        desc=f"Evaluating {target} AutoGluon model"
    )):
        train_subset, eval_subset = feats.iloc[train_index], feats.iloc[test_index]
        predictor = grad_boosting.train_ag(target, train_subset, "autogluon-eval", "good", 500)
        predictions = predictor.predict(eval_subset.drop(columns=targets))

        oof_preds.loc[eval_subset.index, target] = predictions

oof_preds.to_csv("ag_preds.csv")

In [ ]:
grad_boosting.train_ag("Tg", tg_feats)
grad_boosting.train_ag("FFV", ffv_feats)
grad_boosting.train_ag("Tc", tc_feats)
grad_boosting.train_ag("Density", density_feats)
grad_boosting.train_ag("Rg", rg_feats)

masks = {
    "Tg": tg_mask,
    "FFV": ffv_mask,
    "Tc": tc_mask,
    "Density": density_mask,
    "Rg": rg_mask
}
joblib.dump(masks, "../models/autogluon/masks.pkl")